# VAE를 이용한 생성 모델
- Variational Autoencoder은 오코인코더를 개선하여 평균과 표준편차 두개의 성분을 나타내는 두개의 임베딩 벡터를 생성하게 한다
- 이를 이용하면 생성 모델을 확률적으로 조절하기가 수월해진다

<img src="https://github.com/StillWork/image/blob/main/%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202022-11-23%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%204.06.41.png?raw=1" width =600>

- 새로운 분자의 구조를 생성하는 모델에 적용할 수 있다
- 분자 표현으로 SMILES를 사용하며 새로운 SMILES를 얻는다
 - MolrculeNet이 제공하는 SMILES 데이터셋 MUV 사용 (약 90000개 제공)
 - Maximum Unbiased Validation(MUV) - 17개의 태스크 포함
- [VAE 소개](https://towardsdatascience.com/an-introduction-to-variational-auto-encoders-vaes-803ddfb623df)

# import

In [1]:
!pip install DeepChem

In [2]:
!pip install tensorflow==2.9

In [3]:
import deepchem as dc
import tensorflow as tf
import tensorflow.keras.layers as layers

import pandas as pd
import numpy as np
import pickle
%config InlineBackend.figure_format = 'retina'

# 학습 데이터

In [4]:
tasks, datasets, transformers = dc.molnet.load_muv()
train_dataset, valid_dataset, test_dataset = datasets
train_smiles = train_dataset.ids

print(train_smiles[:10])
print(type(train_smiles))
print(len(train_smiles))

['NC(=O)NC(Cc1ccccc1)C(=O)O' 'Nc1ccc(C(=O)O)c(O)c1'
 'C=CCNC(=S)Nc1ccc(Br)cc1F' 'COC(=O)C(NC(=O)Nc1ccccc1F)C(C)C'
 'CC(C)CC(=O)Nc1ccc(OCC(=O)O)cc1' 'CC(NC(=O)c1ccccc1F)C(=O)O'
 'CCOc1ccc(OCCCN(C)C)cc1' 'CCOc1ccc(OCCCCN(C)C)cc1'
 'CCOC(=O)NC(NC(=O)OCC)C(=O)c1ccccc1'
 'CCOC(=O)C(C(=O)OCC)C(=O)c1cc(OC)c(OC)c(OC)c1']
<class 'numpy.ndarray'>
74469


In [5]:
# coconut database 다운로드 후 파일 압축 해제 및 파일명 확인
! wget https://coconut.s3.uni-jena.de/prod/downloads/2024-09/coconut-09-2024.csv.zip
!unzip ./coconut-09-2024.csv.zip

--2024-09-25 14:34:38--  https://coconut.s3.uni-jena.de/prod/downloads/2024-09/coconut-09-2024.csv.zip
Resolving coconut.s3.uni-jena.de (coconut.s3.uni-jena.de)... 141.35.104.25, 141.35.104.26, 2001:638:1558:2368::8d23:681a, ...
Connecting to coconut.s3.uni-jena.de (coconut.s3.uni-jena.de)|141.35.104.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88935618 (85M) [application/zip]
Saving to: ‘coconut-09-2024.csv.zip.2’

coconut-09-2024.csv 100%[===================>]  84.82M  22.3MB/s    in 3.8s    

2024-09-25 14:34:43 (22.3 MB/s) - ‘coconut-09-2024.csv.zip.2’ saved [88935618/88935618]

Archive:  ./coconut-09-2024.csv.zip
replace coconut-09-2024.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
coconut = pd.read_csv('./coconut-09-2024.csv')
coconut

,standard_inchi,standard_inchi_key,canonical_smiles,identifier
0,InChI=1S/C43H53N9O14S2.Na/c1-5-22(3)35-36(57)4...,DRKUXFLLRIKRHH-QDVYGYDXSA-M,CC[C@@H]1NC(=O)[C@@H](NC(=O)[C@H](C)NC(=O)CNC(...,CNP0437004.1
1,InChI=1S/C21H32O12/c1-9-14(23)16(25)18(27)21(3...,OXHVQSRYUNGYOK-NUASCYGXSA-N,COC1=CC=C(CCO[C@@H]2O[C@H](CO[C@@H]3O[C@@H](C)...,CNP0243002.1
2,InChI=1S/C36H61N5O7/c1-21(2)18-27-35(47)48-28(...,NEGZFRNAAJQQEG-NOFCQABOSA-N,C/C1=C\[C@@H](C(C)(C)C)OC(=O)[C@H](CC(C)C)N(C)...,CNP0458114.1
3,InChI=1S/C22H22O9/c1-28-12-4-2-11(3-5-12)15-9-...,DQIVYFNWBDHNFD-WHCFWRGISA-N,COC1=CC=C(C2=CC(=O)OC3=CC(O[C@@H]4O[C@H](CO)[C...,CNP0252086.2
4,InChI=1S/C32H41N5O4/c1-6-18(4)28-32(41)36-12-8...,HKVSEIVDIONNKB-QWNGKRCASA-N,CC[C@H](C)[C@H]1C(=O)N2CCC[C@H]2C(=O)N1C(=O)[C...,CNP0107934.1
...,...,...,...,...
695128,InChI=1S/C19H19N3O4/c1-26-13-8-6-12(7-9-13)20-...,HHSNDFVMRMIDBG-INIZCTEOSA-N,COC1=CC=C(NC(=O)CC[C@@H]2NC(=O)C3=CC=CC=C3NC2=...,CNP0395779.1
695129,InChI=1S/C30H30N2O10/c1-12-23(34)27(38)28(39)3...,VVPODVCQSZKNKL-RLOKSPFPSA-N,CC(=O)OC1=CC=C2C(=O)C3=C(O)C(CC4=CC=CC(C(N)N)=...,CNP0097600.1
695130,InChI=1S/C21H22O7/c1-11(2)4-5-13-15(23)7-6-14(...,LMFCHRAKSGPODM-OAQYLSRUSA-N,COC1=C([C@]2(O)COC3=CC(O)=CC(O)=C3C2=O)C=CC(O)...,CNP0212403.1
695131,InChI=1S/C20H30O7/c1-17(2)4-3-12(23)18-8-27-20...,IJWNAKYUVUUYTE-HMBONYETSA-N,CC1(C)CC[C@H](O)[C@]23COC(O)([C@@H](O)[C@H]12)...,CNP0494455.1


In [7]:
# SMILES와 cid를 저장할 리스트 생성
coconut_smiles = list(coconut['canonical_smiles'])
coconut_cid = list(coconut['identifier'])

print('cid_names :\t', coconut_cid[:3])
print('smiles :\t\t', coconut_smiles[:3])
print('cid_len :\t', len(coconut_cid))
print('smiles_len :\t', len(coconut_smiles))

cid_names :	 ['CNP0437004.1', 'CNP0243002.1', 'CNP0458114.1']
smiles :		 ['CC[C@@H]1NC(=O)[C@@H](NC(=O)[C@H](C)NC(=O)CNC(=O)C2=CC=C(O)C=C2)CNC(=O)[C@H](CS(=O)(=O)[O-])NC(=O)/C=C/C2=CSC(=N2)[C@H](CC2=CC=C(O)C=C2)NC(=O)C(=O)[C@H]([C@@H](C)CC)NC1=O.[Na+]', 'COC1=CC=C(CCO[C@@H]2O[C@H](CO[C@@H]3O[C@@H](C)[C@H](O)[C@@H](O)[C@H]3O)[C@@H](O)[C@H](O)[C@H]2O)C=C1O', 'C/C1=C\\[C@@H](C(C)(C)C)OC(=O)[C@H](CC(C)C)N(C)C(=O)[C@H](C)N(C)C(=O)CNC(=O)[C@H](C(C)C)NC(=O)[C@@H]2CCCN2C(=O)[C@H](C)CC1']
cid_len :	 695133
smiles_len :	 695133


In [8]:
# SMILES 문자열의 규칙을 파악: 문자(토큰)의 목록, 문자열의 최대길이 등

tokens = set()
for s in coconut_smiles[:80000]:
    tokens = tokens.union(set(s))
tokens = sorted(list(tokens))
max_length = max(len(s) for s in coconut_smiles)

In [9]:
tasks, datasets, transformers = dc.molnet.load_muv()
train_dataset, valid_dataset, test_dataset = datasets
train_smiles = train_dataset.ids

# SMILES 문자열의 규칙을 파악: 문자(토큰)의 목록, 문자열의 최대길이 등

tokens = set()
for s in train_smiles:
    tokens = tokens.union(set(s))
tokens = sorted(list(tokens))
max_length = max(len(s) for s in train_smiles)

In [10]:
# SMILES 문자열의 규칙을 파악: 문자(토큰)의 목록, 문자열의 최대길이 등

tokens = set()
for s in train_smiles:
    tokens = tokens.union(set(s))
tokens = sorted(list(tokens))
max_length = max(len(s) for s in train_smiles)

print(tokens)
print(max_length)

['#', '(', ')', '+', '-', '/', '1', '2', '3', '4', '5', '6', '=', 'B', 'C', 'F', 'H', 'N', 'O', 'S', '[', '\\', ']', 'c', 'l', 'n', 'o', 'r', 's']
82


# VAE 모델

- AspuruGuzikAutoEncoder 사용: 인코더는 합성곱신경망을, 디코더는 순환신경망을 사용
- 학습속도를 조절하기 위해서 ExponentialDecay를 사용한다
 - 0.001에서 시작하고 이포크마다 0.95배씩 감소시킨다
- 학습된 모델을 vae 폴더에 저장한다
 - 나중에 모델을 이용하려면 vae 폴더를 구글 드라이브 등에 저장했다가 restore하여 사용한다

In [11]:
from deepchem.models.seqtoseq import AspuruGuzikAutoEncoder
from deepchem.models.optimizers import ExponentialDecay
batch_size = 100
batches_per_epoch = len(train_smiles)/batch_size
learning_rate = ExponentialDecay(0.001, 0.95, batches_per_epoch)
model = AspuruGuzikAutoEncoder(tokens, max_length, model_dir='vae',
                batch_size=batch_size, learning_rate=learning_rate)

# 시퀀스 생성 함수 정의

def generate_sequences(epochs):
    for i in range(epochs):
        print(f'{i} epoch start!')
        for s in train_smiles:
            yield (s, s)

## 모델 학습

In [ ]:
# AspuruGuzikAutoEncoder이 제공_하는 자체 학습 함수 (이포크수 지정)
model.fit_sequences(generatesequences(50)) # 50 이포크 수

0 epoch start!
1 epoch start!
2 epoch start!
3 epoch start!
4 epoch start!
5 epoch start!


## 모델 restore()
- 이미 학습된 모델을 불러오는 방법
- 학습된 모델이 구글 드라이브에 저장되어 있는 경우

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model = AspuruGuzikAutoEncoder(tokens, max_length, model_dir='/content/drive/MyDrive/vae', batch_size=batch_size, learning_rate=learning_rate)
model.restore()

# 분자 생성

- 학습된 모델을 이용하여 새로운 분자를 만든다
- 모델에 들어가는 벡터의 크기를 지정한다 (예: 196)
- 벡터를 2000개 생성하겠다
- 생성된 분자들중 유효한 SMILES를 걸러내기 위해서 RDKit의 MolFromSmiles를 사용한다

In [ ]:
from rdkit import Chem
predictions = model.predict_from_embeddings(np.random.normal(size=(4000,196)))
molecules = []
for p in predictions:
    smiles = ''.join(p)
    if Chem.MolFromSmiles(smiles) is not None:
        molecules.append(smiles)
print()
print('Generated molecules:')
for m in molecules:
    print(m)

In [ ]:
molecules

# 유효한 분자 필터링

- 생성된 SMILES 들에 대해서 유효하지 않거나 약물로서 가치가 없는 분자를 걸러내야 한다
- 제거하고 싶은 분자가 있는지 찾는다
- MolFromSmiles()을 사용해 SMILES 문자열들을 분자 객체로 변환한다
- 분자의 크기를 확인한다 (10보다 작으면 상호작용에 필요한 에너지가 불충분하고, 50 이상이면 분자의 용해도가 너무 낮아 문제가 된다)
- 수소를 제외한 분자의 크기를 GetNumAtoms()로 얻는다
- 약물과 얼마나 유사한지를 판단하기 위해서 QED(Quantitave Estimate of Drugness)를 많이 사용한다
 - QED: 계산된 속성 집합과 판매된 약물의 동일한 특성 분포를 정량화 한 것 (Richard Bickerton 이 제안)
 - 1에 가까울수록 기존의 약물과 유사하다고 본다
 - QED > 0.5 인 분자만 고른 후 결과를 시각화 한다

In [ ]:
from rdkit import Chem
molecules_new = [Chem.MolFromSmiles(x) for x in molecules]
print(sorted(x.GetNumAtoms() for x in molecules_new))

good_mol_list = [x for x in molecules_new if x.GetNumAtoms() > 10 and x.GetNumAtoms() < 50]
print(len(good_mol_list))

In [ ]:
good_mol_list = [x for x in molecules_new if x.GetNumAtoms() > 10 and x.GetNumAtoms() < 50]
print(len(good_mol_list))

In [ ]:
good_mol_list

In [ ]:
from rdkit.Chem import QED
qed_list = [QED.qed(x) for x in good_mol_list]
print(qed_list)
final_mol_list = [(a,b) for a,b in zip(good_mol_list, qed_list) if b > 0.5] #
print(len(final_mol_list))

In [ ]:
qed_list

In [ ]:
final_mol_list = [(a,b) for a,b in zip(good_mol_list, qed_list) if b > 0.5] #
final_mol_list

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
img=Draw.MolsToGridImage([x[0] for x  in final_mol_list],
                         molsPerRow=4,subImgSize=(200,200),
                         legends=[f"{x[1]:.2f}" for x in final_mol_list])
img

In [ ]:
predictions_2 = model.predict_from_embeddings(np.random.normal(size=(10,196)))
molecules_2 = []
for p in predictions_2:
  smiles = ''.join(p)
  # if Chem.MolFromSmiles(smiles) is not None:
  molecules_2.append(smiles)

molecules_2

In [ ]:
np.array(['CC(=O)OC1=CC=CC=C1C(=O)O'])

In [ ]:
print(model.predict_from_sequences(np.array(['CC(=O)OC1=CC=CC=C1C(=O)O'])))

In [ ]:
# aspirin, Penicillin, Morphine, Quinine의 SMILES 데이터를 딕셔너리로 저장
smiles_dict = {'aspirin' : 'CC(=O)OC1=CC=CC=C1C(=O)O',
 'Penicillin' : 'CC1(C(N2C(S1)C(C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C',
 'Morphine' : 'CN1CCC23C4C1CC5=C2C(=C(C=C5)O)OC3C(C=C4)O',
 'Quinine' : 'COC1=CC2=C(C=CN=C2C=C1)C(C3CC4CCN3CC4C=C)O'}

mol_list = []
smiles_list = []
for product in smiles_dict.keys():
    generated = model.predict_from_sequences(np.array([smiles_dict[product]]))
    generated =  ''.join(generated[0])
    generated_mol = Chem.MolFromSmiles(generated)
    if generated_mol is not None:
        print(product)
        print(generated)
        mol_list.append(generated_mol)
        smiles_list.append(generated)

print(mol_list)

In [ ]:
aspirin_mol = Chem.MolFromSmiles(smiles_dict['aspirin'])
mol_img = Draw.MolsToGridImage([aspirin_mol, mol_list[0]],
                                   molsPerRow=2,
                                   subImgSize=(400,200),
                                   legends=['aspirin', 'generated aspirin'])
mol_img